In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
import mediapipe as mp
import numpy as np
import os
import cv2

In [2]:
mp_holistic = mp.solutions.holistic  # Holistic model
mp_drawing = mp.solutions.drawing_utils 

In [3]:
def mediapipe_detection(image, model):
    # Assuming 'image' is your normalized image
    image = (image * 255).astype('uint8')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False  # Image is no longer writeable
    results = model.process(image)  # Make prediction
    image.flags.writeable = True  # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)  # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)  # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS)  # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS)  # Draw right hand connections


In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                              )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                              )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                              )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                              )


In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in
                     results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    face = np.array([[res.x, res.y, res.z] for res in
                     results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)
    lh = np.array([[res.x, res.y, res.z] for res in
                   results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in
                   results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(
        21 * 3)
    return np.concatenate([pose, face, lh, rh])


In [10]:
def process_video_sequence(action_folder, video_path,EXTRACTED_DATA_PATH, sequence, holistic):
    cap = cv2.VideoCapture(video_path)
    frame_num = 0
    frames = []

    while cap.isOpened() and frame_num < 60:
        ret, frame = cap.read()
        if not ret:
            break

        # Normalize pixel values
        frame = frame / 255.0

        frames.append(frame)
        frame_num += 1

        # Make detections
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks
        draw_styled_landmarks(image, results)

        if frame_num == 1:
            cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action_folder, sequence), (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('OpenCV Feed', image)
            cv2.waitKey(500)
        else:
            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action_folder, sequence), (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            # Show to screen
            cv2.imshow('OpenCV Feed', image)

        # Export keypoints
        keypoints = extract_keypoints(results)
        npy_dir = os.path.join(EXTRACTED_DATA_PATH, action_folder, str(sequence))
        os.makedirs(npy_dir, exist_ok=True)  # Ensure the directory exists or create it
        npy_path = os.path.join(npy_dir, str(frame_num - 1) + '.npy')
        np.save(npy_path, keypoints)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
     # Padding frames if necessary to ensure exactly 60 frames
    if frame_num < 60:
        last_frame = frames[-1]
        for i in range(60 - frame_num):
            image, results = mediapipe_detection(last_frame, holistic)
            draw_styled_landmarks(image, results)
            # Show to screen
            cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action_folder, sequence), (15, 12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            cv2.imshow('OpenCV Feed', image)
            cv2.waitKey(1)
            # Export keypoints
            keypoints = extract_keypoints(results)
            npy_dir = os.path.join(EXTRACTED_DATA_PATH, action_folder, str(sequence))
            npy_path = os.path.join(npy_dir, str(frame_num + i) + '.npy')
            np.save(npy_path, keypoints)

    cap.release()
    cv2.destroyAllWindows()

In [11]:
def prepare_data(DATA_PATH, EXTRACTED_DATA_PATH):
    # Set mediapipe model
    holistic_model = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    with holistic_model as holistic:
        # Set the path to your dataset directory
        # DATA_PATH = './Electronics'
        # EXTRACTED_DATA_PATH = './Extracted_Keypoints'  # New directory for extracted keypoints

        # Loop through actions
        actions = os.listdir(DATA_PATH)
        for action_folder in actions:
            # Get the path to the action folder
            action_path = os.path.join(DATA_PATH, action_folder)

            # Loop through video files in the action folder
            video_files = [f for f in os.listdir(action_path) if
                           f.endswith('.MOV') or f.endswith('.mp4')]  # Adjust file extensions as needed
            for idx, video_file in enumerate(video_files):
                # Process each video file as a sequence
                video_path = os.path.join(action_path, video_file)
                sequence = idx
                process_video_sequence(action_folder, video_path, EXTRACTED_DATA_PATH, sequence, holistic)


In [12]:
DATA_PATH = './Electronics'
EXTRACTED_DATA_PATH = './Extracted_Keypoints'  # New directory for extracted keypoints
# Prepare the data
prepare_data(DATA_PATH, EXTRACTED_DATA_PATH)



KeyboardInterrupt: 

In [7]:
# # 6. Preprocess Data and Create Labels and Features
# Path for exported data, numpy arrays
DATA_PATH_1 = os.path.join('Extracted_Keypoints')

# Actions that we try to detect
actions = np.array(['Camera', 'Laptop', 'Radio'])

# Thirty videos worth of data
# no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 60

# Folder start
start_folder = 0


In [8]:
def prepare_dataset(DATA_PATH, actions):
    label_map = {label: num for num, label in enumerate(actions)}
    sequences, labels = [], []
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        for sequence_folder in os.listdir(action_path):
            sequence_length = len(
                [file for file in os.listdir(os.path.join(action_path, sequence_folder)) if file.endswith('.npy')])
            window = []
            for frame_num in range(sequence_length):
                res = np.load(os.path.join(action_path, sequence_folder, "{}.npy".format(frame_num)))
                window.append(res)
            sequences.append(window)
            labels.append(label_map[action])

    X = np.array(sequences)
    y = to_categorical(labels).astype(int)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
    return X_train, X_test, y_train, y_test

In [9]:
X_train, X_test, y_train, y_test = prepare_dataset(DATA_PATH_1, actions)

In [10]:
def build_model(input_shape, num_classes):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(LSTM(256, return_sequences=True, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=False, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [11]:
def compile_and_train(model, X_train, y_train, epochs=500):
    log_dir = os.path.join('Logs')
    tb_callback = TensorBoard(log_dir=log_dir)
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    model.fit(X_train, y_train, epochs=epochs, callbacks=[tb_callback])
    return model

In [12]:
def predict(model, X_test):
    return model.predict(X_test)

In [13]:
def save_model(model, model_path='action.h5'):
    model.save(model_path)

In [ ]:
input_shape = (60, 1662)
num_classes = actions.shape[0]
model = build_model(input_shape, num_classes)
model = compile_and_train(model, X_train, y_train)
predictions = predict(model, X_test)
save_model(model)

C:\Users\91942\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 31s 31s/step - categorical_accuracy: 0.2258 - loss: 16.7400
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step - categorical_accuracy: 0.4194 - loss: 16.0710
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 490ms/step - categorical_accuracy: 0.4194 - loss: 16.3198
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step - categorical_accuracy: 0.4516 - loss: 15.0937
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 465ms/step - categorical_accuracy: 0.3548 - loss: 14.9686
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step - categorical_accuracy: 0.2903 - loss: 14.3624
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 454ms/step - categorical_accuracy: 0.4194 - loss: 14.8898
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step - categorical_accuracy: 0.2581 - loss: 14.1559
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step - categorical_accuracy: 0.4516 - loss: 13.3870
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step - categorical_accuracy: 0.3871 - loss: 13.0998
Epoch 11/500
1/1 ━━━

In [22]:
del model

In [14]:
def build_and_load_model(input_shape, num_classes):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(LSTM(256, return_sequences=True, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=False, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.load_weights('action.h5')
    return model

In [15]:
def evaluate_model(model, X_test, y_test):
    yhat = model.predict(X_test)
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    print(multilabel_confusion_matrix(ytrue, yhat))
    print(accuracy_score(ytrue, yhat))

In [16]:
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85 + num * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)
    return output_frame

In [19]:
def real_time_prediction(model, actions, input_shape, threshold=0.2):
    sequence = []
    sentence = []
    predictions = []
    colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]
    cap = cv2.VideoCapture(0)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            frame = frame/255
            image, results = mediapipe_detection(frame, holistic)
            # image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
            draw_styled_landmarks(image, results)
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]
            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                predictions.append(np.argmax(res))
                if np.unique(predictions[-10:])[0] == np.argmax(res):
                    if res[np.argmax(res)] > threshold:
                        if len(sentence) > 0:
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])
                if len(sentence) > 5:
                    sentence = sentence[-5:]
                image = prob_viz(res, actions, image, colors)
                cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
                cv2.putText(image, ' '.join(sentence), (3, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
        cap.release()
        cv2.destroyAllWindows()
    print(' '.join(sentence))

In [20]:
input_shape = (29, 1662)
num_classes = actions.shape[0]
model = build_and_load_model(input_shape, num_classes)
evaluate_model(model, X_test, y_test)
real_time_prediction(model, actions, input_shape)

C:\Users\91942\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 788ms/step
[[[0 1]
  [0 1]]

 [[1 0]
  [1 0]]]
0.5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 855ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━